In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import norm, chi2, f
import statsmodels.api as sm
import statsmodels.stats.api as sms
import statsmodels.formula.api as smf
from statsmodels.graphics import regressionplots
from statsmodels.stats.multicomp import MultiComparison
from statsmodels.robust.robust_linear_model import RLM
from statsmodels.iolib.summary2 import summary_col
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.diagnostic import linear_reset
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from mlxtend.feature_selection import ExhaustiveFeatureSelector as EFS
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.ensemble import RandomForestRegressor
from boruta import BorutaPy
from statsmodels.stats.diagnostic import het_breuschpagan, het_white, het_goldfeldquandt
from sklearn.impute import KNNImputer
from statsmodels.imputation import mice

#these are saved import codes, not all are necessary. 

In [2]:
fdata = pd.read_csv('five_factor_data.csv', skiprows=4)
fdata = fdata[pd.to_numeric(fdata['Unnamed: 0'], errors='coerce').notna()]
fdata = fdata.rename(columns = {'Unnamed: 0': 'date', 'Mkt-RF':'MKTRF'})
fdata = fdata.set_index('date')
fdata

,MKTRF,SMB,HML,RMW,CMA,RF
date,,,,,,
196307,-0.39,-0.48,-0.81,0.64,-1.15,0.27
196308,5.08,-0.80,1.70,0.40,-0.38,0.25
196309,-1.57,-0.43,0.00,-0.78,0.15,0.27
196310,2.54,-1.34,-0.04,2.79,-2.25,0.29
196311,-0.86,-0.85,1.73,-0.43,2.27,0.27
...,...,...,...,...,...,...
2020,23.65,3.74,-46.94,-5.21,-11.70,0.44
2021,23.87,-1.02,25.61,26.72,11.67,0.04
2022,-21.27,-1.57,25.68,6.55,22.50,1.42


In [3]:
data55 = pd.read_csv('data55.csv', skiprows=15)
data55 = data55.rename(columns = {'Unnamed: 0':'date'})
data55 = data55.set_index('date')
data55 = data55[:-1]
data55

,SMALL LoBM,ME1 BM2,ME1 BM3,ME1 BM4,SMALL HiBM,ME2 BM1,ME2 BM2,ME2 BM3,ME2 BM4,ME2 BM5,...,ME4 BM1,ME4 BM2,ME4 BM3,ME4 BM4,ME4 BM5,BIG LoBM,ME5 BM2,ME5 BM3,ME5 BM4,BIG HiBM
date,,,,,,,,,,,,,,,,,,,,,
192607,5.8276,-1.7006,0.5118,-2.1477,1.9583,1.2118,2.4107,0.6056,-2.6082,-0.4527,...,1.5376,1.5460,1.3389,0.2765,2.4678,3.3248,6.0909,2.0285,3.1263,0.5623
192608,-2.0206,-8.0282,1.3968,2.1483,8.5104,2.3620,-0.7525,3.8984,0.2299,6.2937,...,1.3858,3.8587,1.9738,2.1336,5.3422,1.0169,4.1975,1.9769,5.4924,7.7576
192609,-4.8291,-2.6806,-4.3417,-3.2683,0.8586,-2.6849,-0.5252,1.0789,-3.2877,-0.9419,...,1.6897,-0.5246,-1.7724,1.4806,0.8730,-1.2951,3.6610,0.1384,-0.7497,-2.4284
192610,-9.3633,-3.5519,-3.5024,3.4413,-2.5452,-2.8014,-4.4191,-5.0767,-8.0271,-1.3213,...,-3.9136,-2.6528,-2.1058,-3.2532,-5.3525,-2.7382,-3.0061,-2.2467,-4.6725,-5.8129
192611,5.5888,4.1877,2.4384,-4.4495,0.5110,3.1023,-1.7317,3.0425,4.9538,2.7292,...,3.4492,2.3823,3.7315,5.1102,1.8213,4.4331,2.5355,1.5280,3.6596,2.5636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202505,0.0375,0.0688,0.0128,0.0387,0.0025,0.1647,0.1084,0.0696,0.0590,0.0893,...,0.0767,0.0393,0.0512,0.1279,0.0312,0.0751,0.0711,0.0568,0.0685,0.0340
202506,0.0539,0.0704,0.0161,0.0413,-0.0022,0.1653,0.1108,0.0664,0.0615,0.0773,...,0.0745,0.0389,0.0512,0.1255,0.0312,0.0743,0.0731,0.0552,0.0665,0.0340
202507,0.1489,0.0678,0.0767,-0.0169,0.0142,0.1367,0.1589,0.0925,0.0138,0.0155,...,0.1284,0.0531,0.0271,0.1640,0.0214,0.1769,0.0543,0.0521,0.0420,-0.0818


In [4]:
data = pd.merge(data55, fdata, on='date', how='inner')
data

,SMALL LoBM,ME1 BM2,ME1 BM3,ME1 BM4,SMALL HiBM,ME2 BM1,ME2 BM2,ME2 BM3,ME2 BM4,ME2 BM5,...,ME5 BM2,ME5 BM3,ME5 BM4,BIG HiBM,MKTRF,SMB,HML,RMW,CMA,RF
date,,,,,,,,,,,,,,,,,,,,,
196307,1.1287,-0.3632,0.7223,-0.0413,-1.2447,-1.8076,0.1929,-1.0149,-1.9749,-1.1880,...,0.4839,1.1360,-0.4285,-1.1045,-0.39,-0.48,-0.81,0.64,-1.15,0.27
196308,4.2396,1.3730,1.4917,2.5068,4.6644,5.5703,4.5220,4.4450,4.4662,8.2451,...,4.2633,4.6341,8.1704,6.3984,5.08,-0.80,1.70,0.40,-0.38,0.25
196309,-1.7343,0.6204,-1.0007,-1.5215,-0.3584,-4.0525,-1.5072,-0.8638,-1.4935,-2.9243,...,-0.8081,-0.8497,-0.1912,-3.5033,-1.57,-0.43,0.00,-0.78,0.15,0.27
196310,0.3778,-0.7329,1.3066,0.1904,2.3711,1.1926,4.2411,2.3526,2.3058,3.9314,...,1.7420,-0.3354,2.4176,0.4702,2.54,-1.34,-0.04,2.79,-2.25,0.29
196311,-3.3319,-3.8436,-1.7893,-1.0535,-1.1077,-4.2596,-1.7484,-0.7845,-0.0554,-0.1150,...,1.0080,-1.6914,-2.1316,1.3496,-0.86,-0.85,1.73,-0.43,2.27,0.27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202505,0.0375,0.0688,0.0128,0.0387,0.0025,0.1647,0.1084,0.0696,0.0590,0.0893,...,0.0711,0.0568,0.0685,0.0340,6.06,-0.72,-2.88,1.29,2.51,0.38
202506,0.0539,0.0704,0.0161,0.0413,-0.0022,0.1653,0.1108,0.0664,0.0615,0.0773,...,0.0731,0.0552,0.0665,0.0340,4.86,-0.02,-1.60,-3.20,1.45,0.34
202507,0.1489,0.0678,0.0767,-0.0169,0.0142,0.1367,0.1589,0.0925,0.0138,0.0155,...,0.0543,0.0521,0.0420,-0.0818,1.98,-0.15,-1.27,-0.29,-2.08,0.34


## 2a. Data Sources

All data for this replication project comes from the **Kenneth R. French Data Library**, the public data archive maintained by the co-author of the Five-Factor Asset Pricing Model. The library provides the exact factor and portfolio files used in Fama and French (2015), already cleaned and formatted by the authors. Because the data is published directly by the original source, no external cleaning or reconstruction was required.

### Five-Factor Model Data
The five factor series—Mkt-RF, SMB, HML, RMW, CMA, and RF—was downloaded from the:

**Kenneth R. French Data Library**  
https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html

We use the **Changes in CRSP Data** monthly dataset, which contains the market excess return and the four additional factors introduced in the paper. The file covers data from July, 1963 onward and is updated regularly by the authors.

### Test Portfolio Data
To replicate the empirical setup in the original study, we also use the **`25 Portfolios Formed on Size and Book-to-Market (5x5)`** dataset from the same website. These portfolios are constructed using two-way sorts on firm size and book-to-market ratios and are the standard test assets used in evaluating factor models.

### Structure of the Author-Provided Files
Both datasets follow a consistent CSV format:

- Several descriptive header lines  
- A date column labeled "Unnamed: 0" containing YYYYMM date identifiers  
- Monthly percentage returns for each factor or portfolio  
- A copyright footer line  

Since these files are prepared by the authors themselves, they require almost no cleaning beyond removing the non-data header and footer lines. After these are removed, the files load correctly and can be merged using the date column.

## 2b. Data Processing

Because the datasets are published directly by Kenneth French, they require minimal cleaning. The main steps involved removing descriptive header and footer rows, renaming the date column, setting the date as the index, and merging the factor data with the portfolio returns so the regression could be estimated.

### Preparing the Five-Factor File
The five-factor CSV contains several descriptive header lines followed by a numerical dataset, along with a copyright footer at the bottom. These non-data rows were removed on import. The date column appears as "Unnamed: 0" in the raw file, so it was renamed to "date" and used as the index.

```python
fdata = pd.read_csv('five_factor_data.csv', skiprows=4)
fdata = fdata[pd.to_numeric(fdata['Unnamed: 0'], errors='coerce').notna()]
fdata = fdata.rename(columns={'Unnamed: 0': 'date', 'Mkt-RF': 'MKTRF'})
fdata = fdata.set_index('date')
```
### Preparing the 25 Portfolios (5x5) File

The file containing the **25 Size–Book-to-Market portfolios (5x5)** is stored locally as **`data55.csv`**. This dataset includes 25 portfolio return series created from two-way sorts on firm size and book-to-market ratios. Similar to the factor file, it includes a long descriptive header and a non-numerical footer line.

These rows were removed by skipping the header on import and slicing off the final line. The date column was again labeled "Unnamed: 0" in the raw file and was renamed for consistency before setting it as the index.

```python
data55 = pd.read_csv('data55.csv', skiprows=15)
data55 = data55.rename(columns={'Unnamed: 0': 'date'})
data55 = data55.set_index('date')
data55 = data55[:-1]
```

This results in a clean monthly return series for all 25 portfolios, indexed by the same YYYYMM date format used in the factor dataset.

### Merging the Data

Once both datasets were cleaned and indexed by date, they were merged using an inner join. This ensures that each portfolio return observation is matched with the corresponding month’s factor values from the five-factor model.

```python
data = pd.merge(data55, fdata, on='date', how='inner')
```

The merged dataset contains the full set of portfolio returns along with the market, size, value, profitability, and investment factors. This final table is the basis for the regression analysis presented in Section 3.